In [2]:
using Random

bitHDV(N::Int=10000) = bitrand(N)

function bitadd(vectors::BitVector ...)
    v = reduce(.+, vectors)
    n = length(vectors) / 2
    x = [i > n ? 1 : i < n ? 0 : rand(0:1) for i in v]
    return convert(BitVector, x)
end

bitbind(vectors::BitVector ...) =  reduce(.⊻, vectors)

bitperm(vector::BitVector, k::Int=1) = circshift(vector, k)

hamming(x::BitVector, y::BitVector) = sum(x .!= y)/length(x)

hamming (generic function with 1 method)

In [4]:
using DataFrames, CSV

In [7]:
data = CSV.read("/home/mfat/Unief/Thesis/ThesisFatjanov/notebooks/zoo.csv", DataFrame)

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed
,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,aardvark,1,0,0,1,0,0,1,1
2,antelope,1,0,0,1,0,0,0,1
3,bass,0,0,1,0,0,1,1,1
4,bear,1,0,0,1,0,0,1,1
5,boar,1,0,0,1,0,0,1,1
6,buffalo,1,0,0,1,0,0,0,1
7,calf,1,0,0,1,0,0,0,1
8,carp,0,0,1,0,0,1,0,1
9,catfish,0,0,1,0,0,1,1,1
